In [3]:
from collections import deque
from pprint import pprint
import graphviz

In [5]:
def bsuccessors(state):
    """Return a dict of {state: action} pairs. A state is a (here, there, t)
    tuple, where here and there are frozensets of people (indicated by their
    times) and/or the 'light', and t is a number indicating the elapsed time.
    Action is represented by '->' for here to there and '<-' for there to
    here."""
    here, there, t = state
    if 'light' in here:
        return dict(((here - frozenset([a, b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a, b)),
                      (a, b, '->'))
                      for a in here if a != 'light'
                      for b in here if b != 'light')
    else:
        return dict(((here | frozenset([a, b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a, b)),
                      (a, b, '<-'))
                      for a in there if a != 'light'
                      for b in there if b != 'light')
            
initial_state = (frozenset({1, 2, 5, 10, 'light'}), frozenset(), 0)
pprint(bsuccessors(initial_state))

{(frozenset({10, 5}), frozenset({1, 2, 'light'}), 2): (2, 1, '->'),
 (frozenset({2, 10}), frozenset({1, 5, 'light'}), 5): (5, 1, '->'),
 (frozenset({2, 5}), frozenset({1, 10, 'light'}), 10): (10, 1, '->'),
 (frozenset({2, 10, 5}), frozenset({1, 'light'}), 1): (1, 1, '->'),
 (frozenset({1, 10}), frozenset({2, 5, 'light'}), 5): (5, 2, '->'),
 (frozenset({1, 5}), frozenset({2, 10, 'light'}), 10): (10, 2, '->'),
 (frozenset({1, 10, 5}), frozenset({2, 'light'}), 2): (2, 2, '->'),
 (frozenset({1, 2}), frozenset({10, 5, 'light'}), 10): (10, 5, '->'),
 (frozenset({1, 2, 10}), frozenset({5, 'light'}), 5): (5, 5, '->'),
 (frozenset({1, 2, 5}), frozenset({10, 'light'}), 10): (10, 10, '->')}


In [7]:
def bridge_problem(here):
    dot = graphviz.Digraph('Bridge Problem')
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    frontier = [[(here, frozenset(), 0)]] # ordered list of paths we have blazed
    dot.node(str((here, frozenset(), 0)))
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        here1, there1, t1 = state1 = path[-1]
        if not here1 or here1 == set(['light']):
            return dot
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                dot.edge(str(path[-1]), str(state), label=str(action))
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
                frontier.append(path2)
                frontier.sort(key=elapsed_time)
    return []

def elapsed_time(path):
    return path[-1][2]

dot = bridge_problem([1, 2, 5, 10])
dot